## Import libraries

In [19]:
import pandas as pd # Preliminary Wrangling
import numpy as np  # for some data manipulation

# visualization libraries
import matplotlib.pyplot as plt 
import seaborn as sn 

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## read the dataset


In [2]:
def read_csv(filepath):
    """the function to read the dataset 
    
    args
    ----
    
    filepath: path of the data 
    
    returns
    ------
    df: dataframe
    
    
    """   
    try:
        df = pd.read_csv(filepath)
        return df 
    
    except FileNotFoundError:
            print('this is file is not found')

In [3]:
data = read_csv('../Data/prosperLoanData.csv')

# Data Wrangling 

## dataset information 

In [4]:
# the shape of the dataset 

data.shape

(113937, 81)

In [5]:
### display first 5 rows of the data
data.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [6]:
# display dataset information 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

We have different types of columns including integer, float, bool and objects in our datasets 

In [8]:
# check for unique values 
data.nunique().sort_values(ascending=False).head(25)

ListingKey                         113066
LoanNumber                         113066
LoanKey                            113066
ListingNumber                      113066
ListingCreationDate                113064
DateCreditPulled                   112992
MemberKey                           90831
LP_InterestandFees                  78751
LP_CustomerPayments                 78523
LP_CustomerPrincipalPayments        49828
RevolvingCreditBalance              38555
AvailableBankcardCredit             31829
MonthlyLoanPayment                  23567
LP_ServiceFees                      19813
LP_GrossPrincipalLoss               15848
LP_NetPrincipalLoss                 15709
ProsperPrincipalOutstanding         13875
StatedMonthlyIncome                 13502
FirstRecordedCreditLine             11585
AmountDelinquent                     7059
BorrowerAPR                          6677
LP_CollectionFees                    6175
OpenRevolvingMonthlyPayment          2834
ClosedDate                        

In [11]:
# missing values 

def missing_values(df):
    """function to find columns with high number of null values 
    
    Args
    ----
    
    df(dataframe): our dataset
    
    returns
    -------
    returns a list of colummns with null values greater that 30 percent 
    
    """
    list_of_columns_high_null_values = []
    for columns,values in df.isna().sum().items():
        perc_null_values = 100 * (values/df.shape[0])
        if perc_null_values > 30:
            list_of_columns_high_null_values.append(columns)
    
    return list_of_columns_high_null_values

In [13]:
# columns with high null values 

missing_values(data)

['CreditGrade',
 'ClosedDate',
 'GroupKey',
 'TotalProsperLoans',
 'TotalProsperPaymentsBilled',
 'OnTimeProsperPayments',
 'ProsperPaymentsLessThanOneMonthLate',
 'ProsperPaymentsOneMonthPlusLate',
 'ProsperPrincipalBorrowed',
 'ProsperPrincipalOutstanding',
 'ScorexChangeAtTimeOfListing',
 'LoanFirstDefaultedCycleNumber']

## cleaning the dataset 

After evaluate the dataset, I found different observations like column with high number of null values and high cardinality columns. Therefore the first steps is to extract columns I will be using making my analysis



In [14]:
# columns of interest 

used_columns = ['ListingCreationDate','Term', 'LoanStatus', 'BorrowerAPR','BorrowerRate', 'ProsperRating (Alpha)','ProsperScore', 'ListingCategory (numeric)','BorrowerState','EmploymentStatus',
    'IsBorrowerHomeowner','DelinquenciesLast7Years', 'StatedMonthlyIncome', 'TotalProsperLoans', 'LoanOriginalAmount',
    'LoanOriginationDate', 'Recommendations', 'Investors']

In [16]:
## extract those columns 
data_cols_used = data[used_columns]

In [17]:
data_cols_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ListingCreationDate        113937 non-null  object 
 1   Term                       113937 non-null  int64  
 2   LoanStatus                 113937 non-null  object 
 3   BorrowerAPR                113912 non-null  float64
 4   BorrowerRate               113937 non-null  float64
 5   ProsperRating (Alpha)      84853 non-null   object 
 6   ProsperScore               84853 non-null   float64
 7   ListingCategory (numeric)  113937 non-null  int64  
 8   BorrowerState              108422 non-null  object 
 9   EmploymentStatus           111682 non-null  object 
 10  IsBorrowerHomeowner        113937 non-null  bool   
 11  DelinquenciesLast7Years    112947 non-null  float64
 12  StatedMonthlyIncome        113937 non-null  float64
 13  TotalProsperLoans          22

## convert type of the some columns

It can be seen that some columns have the wrong data type for instance `ListingCreationDate` which is object and is supposed to be datetime and others

In [22]:
# columns data type conversion 

data_cols_used['ProsperScore'] = data_cols_used['ProsperScore'].astype('category')
data_cols_used['TotalProsperLoans'] = data_cols_used['TotalProsperLoans'].fillna(0).astype('int64')
data_cols_used['ListingCreationDate'] = pd.to_datetime(data_cols_used['ListingCreationDate'])
data_cols_used['LoanOriginationDate'] = pd.to_datetime(data_cols_used['LoanOriginationDate'])

In [23]:
data_cols_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ListingCreationDate        113937 non-null  datetime64[ns]
 1   Term                       113937 non-null  int64         
 2   LoanStatus                 113937 non-null  object        
 3   BorrowerAPR                113912 non-null  float64       
 4   BorrowerRate               113937 non-null  float64       
 5   ProsperRating (Alpha)      84853 non-null   object        
 6   ProsperScore               84853 non-null   category      
 7   ListingCategory (numeric)  113937 non-null  int64         
 8   BorrowerState              108422 non-null  object        
 9   EmploymentStatus           111682 non-null  object        
 10  IsBorrowerHomeowner        113937 non-null  bool          
 11  DelinquenciesLast7Years    112947 non-null  float64 

In [24]:
data_cols_used.isna().sum()

ListingCreationDate              0
Term                             0
LoanStatus                       0
BorrowerAPR                     25
BorrowerRate                     0
ProsperRating (Alpha)        29084
ProsperScore                 29084
ListingCategory (numeric)        0
BorrowerState                 5515
EmploymentStatus              2255
IsBorrowerHomeowner              0
DelinquenciesLast7Years        990
StatedMonthlyIncome              0
TotalProsperLoans                0
LoanOriginalAmount               0
LoanOriginationDate              0
Recommendations                  0
Investors                        0
dtype: int64

though our dataset has some missing values but we can omit that and continue our analysis with the remaining data

In [26]:
## summary statistics 
data_cols_used.describe()

,Term,BorrowerAPR,BorrowerRate,ListingCategory (numeric),DelinquenciesLast7Years,StatedMonthlyIncome,TotalProsperLoans,LoanOriginalAmount,Recommendations,Investors
count,113937.000000,113912.000000,113937.000000,113937.000000,112947.000000,1.139370e+05,113937.000000,113937.00000,113937.000000,113937.000000
mean,40.830248,0.218828,0.192764,2.774209,4.154984,5.608026e+03,0.275459,8337.01385,0.048027,80.475228
std,10.436212,0.080364,0.074818,3.996797,10.160216,7.478497e+03,0.654774,6245.80058,0.332353,103.239020
min,12.000000,0.006530,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1000.00000,0.000000,1.000000
25%,36.000000,0.156290,0.134000,1.000000,0.000000,3.200333e+03,0.000000,4000.00000,0.000000,2.000000
50%,36.000000,0.209760,0.184000,1.000000,0.000000,4.666667e+03,0.000000,6500.00000,0.000000,44.000000
75%,36.000000,0.283810,0.250000,3.000000,3.000000,6.825000e+03,0.000000,12000.00000,0.000000,115.000000
max,60.000000,0.512290,0.497500,20.000000,99.000000,1.750003e+06,8.000000,35000.00000,39.000000,1189.000000


## Save cleaned data 

In [27]:
data_cols_used.to_csv('../Data/Cleaned_data.csv',index=False)